In [1]:
import os
import midas_touch2 as mt2
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
import matplotlib.pyplot as plt
import stock_mapping as sm
import warnings
import asyncio
import time
from datetime import datetime
from class_mapping import StockSymbolMapper
pd.options.display.max_columns = 100000
warnings.filterwarnings('ignore')

if not mt5.initialize():
    print("initialize() failed")
else:
    print("MT5 successfully initialised.\n")

MT5 successfully initialised.



In [2]:
yf_symbols = ['0166.KL', '2062.KL', '2488.KL', '3336.KL', '4677.KL', '5031.KL',
              '5106.KL', '5138.KL', '5200.KL', '7029.KL', '7090.KL', '7216.KL',
              '7277.KL']
symbols = [symbol + "SE" for symbol in yf_symbols]
api_token = "x"
data = mt2.download_stocks(symbols, "d", api_token, num_days=20)
price_df = pd.DataFrame(data)
#rename columns
file_path = r'C:\Users\kkyao\midas_touch2\stock_mapping.xlsx'
mt5_symbol = sm.get_ticker_list(yf_symbols, file_path=file_path)
price_df.columns = mt5_symbol
#price_df.columns = yf_symbols
weights_df, latest_weights = mt2.run_portfolio(price_df, alpha_n=5)
price_df.tail()

,INARI,HARBOUR,ABMB,IJM,YTL,TIMECOM,AXREIT,HSPLANT,UOADEV,MASTER,AHEALTH,KAWAN,DIALOG
time,,,,,,,,,,,,,
2023-11-06,2.98,1.19,3.45,1.89,1.5097,5.26,1.7786,1.81,1.73,2.89,2.42,1.84,2.14
2023-11-07,2.97,1.18,3.42,1.87,1.5097,5.23,1.7885,1.80,1.72,2.90,2.44,1.85,2.13
2023-11-08,2.95,1.18,3.40,1.91,1.5000,5.22,1.7885,1.80,1.73,2.86,2.46,1.85,2.13
2023-11-09,2.96,1.18,3.38,1.90,1.5300,5.18,1.7700,1.80,1.72,2.88,2.48,1.85,2.11
2023-11-10,2.92,1.19,3.38,1.89,1.5300,5.16,1.7800,1.79,1.73,2.96,2.43,1.87,2.10


In [3]:
latest_weights

INARI      0.042670
HARBOUR    0.000000
ABMB       0.182249
IJM        0.000000
YTL        0.000000
TIMECOM    0.284142
AXREIT     0.237036
HSPLANT    0.069457
UOADEV     0.000000
MASTER     0.000000
AHEALTH    0.000000
KAWAN      0.066501
DIALOG     0.117946
Name: 2023-11-10, dtype: float64

In [4]:
# file_path = r'C:\Users\kkyao\midas_touch2\stock_mapping.xlsx'
# mapper = StockSymbolMapper(file_path)
# output = mapper.find_opposite(yf_symbols)
# print(output)

In [19]:
###dummy weights to test
import random
def generate_random_series(tickers, seed=None):
    if seed is not None:
        random.seed(seed)
    random_values = [random.random() for _ in tickers]
    total = sum(random_values)
    normalized_values = [value / total for value in random_values]
    return pd.Series(normalized_values, index=tickers)
latest_weights = generate_random_series(mt5_symbol)
latest_weights

INARI      0.102864
HARBOUR    0.123454
ABMB       0.069812
IJM        0.083728
YTL        0.045091
TIMECOM    0.005686
AXREIT     0.062302
HSPLANT    0.071819
UOADEV     0.117415
MASTER     0.094135
AHEALTH    0.009003
KAWAN      0.114251
DIALOG     0.100440
dtype: float64

In [21]:
total_investment = 100000 #TRY 15K 10K 5K
previous_allocation_df = mt2.get_folder(folder_name='daily_allocation')
previous_portfolio_df = mt2.get_folder(folder_name='daily_portfolio')
if previous_allocation_df is not None and previous_portfolio_df is not None:
    # Process the existing CSV data
    prev_holding = previous_allocation_df['Holding Units']
    prev_entry = previous_allocation_df['Entry Units']
    prev_exit = previous_allocation_df['Exit Units']
    total_investment = abs(previous_portfolio_df['Total Value'].iloc[-1])
    allocation_df = mt2.calculate_stock_allocation(total_investment, latest_weights, price_df)
    allocation_df['Holding Units'] = prev_holding + prev_entry - prev_exit
else: #first run
    allocation_df = mt2.calculate_stock_allocation(total_investment, latest_weights, price_df)
    # Handle the initial run scenario
    print("This is the initial run. No previous allocation data available.")
    allocation_df['Holding Units'] = 0

# Calculate Entry and Exit Units
allocation_df['Entry Units'] = allocation_df.apply(
    lambda row: max(row['Allocated Units'] - row['Holding Units'], 0), axis=1
)
allocation_df['Exit Units'] = allocation_df.apply(
    lambda row: max(row['Holding Units'] - row['Allocated Units'], 0), axis=1
)
numeric_cols = allocation_df.select_dtypes(include=['number']).astype(float)

for col in numeric_cols.columns:
    allocation_df[col] = numeric_cols[col]

########################---async start here---#########################
print(allocation_df)
order_ids = mt2.execute_trades_from_data(allocation_df)
time.sleep(5)
orders_list = [mt2.check_order_status(order_id) for order_id in order_ids]
print(f'orders_list: {orders_list}')
deleted_volumes = mt2.delete_orders(orders_list)
print(f'deleted_volumes: {deleted_volumes}')
########################---end here----#################################
for order in deleted_volumes:
    symbol = order['symbol']
    volume_filled = order['volume_filled']
    if order['direction'] == mt5.ORDER_TYPE_BUY_LIMIT:
        allocation_df.loc[allocation_df['Share Symbol'] == symbol, 'Entry Units'] = volume_filled
    else: 
        allocation_df.loc[allocation_df['Share Symbol'] == symbol, 'Exit Units'] = volume_filled

    #allocation_df.loc[allocation_df['Share Symbol'] == symbol, 'Allocated Units'] = volume_filled
# async def execute_and_monitor(allocation_df, timeout=60):
#     print(allocation_df)
#     order_ids = mt2.execute_trades_from_data(allocation_df)
#     print(f'Waiting for {timeout/60} minutes..')
#     await asyncio.sleep(timeout)
#     orders_list = [mt2.check_order_status(order_id) for order_id in order_ids]  # Renamed from orders_list
#     print(f'orders_list: {orders_list}')
#     resubmitted_orders = mt2.delete_and_resubmit_orders(orders_list)  # Renamed from resubmitted_orders
#     print(f'resubmitted_orders: {resubmitted_orders}')
#     return orders_list, resubmitted_orders  # Return the renamed variables

# # Execute the async function and capture the returned values with the new variable names
# orders_list, resubmitted_orders = asyncio.run(execute_and_monitor(allocation_df, timeout=90))

# Calculate the portfolio value
entry_unit = allocation_df['Entry Units']
exit_unit = allocation_df['Exit Units']
share_price = allocation_df['Share Price']
allocated_unit = allocation_df['Allocated Units']
if previous_allocation_df is not None and previous_portfolio_df is not None: #2nd run onwards
    starting_cash = previous_portfolio_df['Cash Value'].iloc[-1]
    current_share_value = ((entry_unit*share_price)-(exit_unit*share_price)).sum()
    current_share_value += previous_portfolio_df['Portfolio Value'].iloc[-1]

else: #first run
    starting_cash = total_investment
    current_share_value = (entry_unit*share_price).sum()

portfolio_df = mt2.compile_portfolio_data(allocation_df, starting_cash, current_share_value)

mt2.save_df_to_csv(portfolio_df, 
                   folder_name='daily_portfolio', 
                   file_name='balance',
                   append=True)
mt2.save_df_to_csv(allocation_df, 
                   folder_name='daily_allocation', 
                   file_name='allocation')


   Share Symbol  Share Price  Allocated Units   Weights  Holding Units  \
0         INARI         2.92           3500.0  0.102864         3500.0   
1       HARBOUR         1.19          10300.0  0.123454        10300.0   
2          ABMB         3.38           2000.0  0.069812         2000.0   
3           IJM         1.89           4400.0  0.083728         4400.0   
4           YTL         1.53           2900.0  0.045091         2900.0   
5       TIMECOM         5.16            100.0  0.005686          100.0   
6        AXREIT         1.78           3500.0  0.062302         3500.0   
7       HSPLANT         1.79           4000.0  0.071819         4000.0   
8        UOADEV         1.73           6700.0  0.117415         6700.0   
9        MASTER         2.96           3100.0  0.094135         3100.0   
10      AHEALTH         2.43            300.0  0.009003          300.0   
11        KAWAN         1.87           6100.0  0.114251         6100.0   
12       DIALOG         2.10          

In [7]:
deleted_volumes

[]

In [8]:
print(f'starting_cash: {starting_cash}')
print(f'current_share_value: {current_share_value}')

starting_cash: 837.0
current_share_value: 99163.0


In [9]:
portfolio_df

,Starting Capital,Purchase Cost,Sale Proceed,Cash Value,Portfolio Value,Total Value
0,837.0,0.0,0.0,837.0,99163.0,100000.0


In [10]:
allocation_df

,Share Symbol,Share Price,Allocated Units,Weights,Holding Units,Entry Units,Exit Units
0,INARI,2.92,1400.0,0.042670,1400.0,0.0,0.0
1,HARBOUR,1.19,0.0,0.000000,0.0,0.0,0.0
2,ABMB,3.38,5300.0,0.182249,5300.0,0.0,0.0
3,IJM,1.89,0.0,0.000000,0.0,0.0,0.0
4,YTL,1.53,0.0,0.000000,0.0,0.0,0.0
5,TIMECOM,5.16,5500.0,0.284142,5500.0,0.0,0.0
6,AXREIT,1.78,13300.0,0.237036,13300.0,0.0,0.0
7,HSPLANT,1.79,3800.0,0.069457,3800.0,0.0,0.0
8,UOADEV,1.73,0.0,0.000000,0.0,0.0,0.0
9,MASTER,2.96,0.0,0.000000,0.0,0.0,0.0


In [11]:
price_df

,INARI,HARBOUR,ABMB,IJM,YTL,TIMECOM,AXREIT,HSPLANT,UOADEV,MASTER,AHEALTH,KAWAN,DIALOG
time,,,,,,,,,,,,,
2023-10-23,2.84,1.14,3.35,1.82,1.2273,5.23,1.7944,1.84,1.68,2.80,2.42,1.84,2.13
2023-10-24,2.92,1.16,3.35,1.86,1.2760,5.25,1.8044,1.84,1.69,2.80,2.41,1.84,2.17
2023-10-25,2.94,1.16,3.35,1.86,1.2955,5.29,1.7944,1.85,1.70,2.79,2.41,1.87,2.16
2023-10-26,2.87,1.16,3.36,1.90,1.3636,5.25,1.7845,1.84,1.70,2.78,2.42,1.87,2.14
2023-10-27,2.82,1.17,3.37,1.90,1.3734,5.22,1.7944,1.83,1.70,2.82,2.43,1.87,2.15
2023-10-30,2.83,1.17,3.38,1.89,1.3539,5.22,1.7944,1.83,1.68,2.77,2.42,1.87,2.15
2023-10-31,2.85,1.16,3.43,1.90,1.4221,5.32,1.8044,1.79,1.69,2.77,2.41,1.87,2.15
2023-11-01,2.82,1.16,3.40,1.87,1.4416,5.25,1.7944,1.80,1.69,2.80,2.44,1.86,2.06
2023-11-02,2.89,1.17,3.41,1.88,1.5000,5.26,1.8044,1.80,1.71,2.83,2.42,1.85,2.08
